# Inferring the coupling between two biological oscillators

![image coupling](https://colasdroin.github.io/images/coupling.png)

## Context and output

I have worked on this project from 2016 to 2018, during the first two years of my PhD. It was my main project, and I was basically responsible for everything (theoretical model, computational analysis) but the biological experiments. My efforts have paid, as this led to a [publication in _Nature Physics_](https://www.nature.com/articles/s41567-019-0598-1).

The code, fully open-source, is available [here](https://github.com/ColasDroin/CouplingHMM). 

## Simplified abstract

This interdisciplinary physics project, published in 2019 in Nature Physics, builds upon previous work from my lab aimed at understanding how two seemingly unrelated biological oscillators, the cell cycle and the circadian clock, turn out to robustly synchronize under various environmental conditions. This system is very attractive for physicists and quantitative biologists since it’s both sufficiently simple, i.e. ‘lives’ on a low dimensional manifold, yet complex enough to exhibit universal dynamics. We decided to use data-driven model reconstruction and non-linear dynamics analysis to investigate the low-dimensional behaviour of the coupled oscillators system, in both mouse and human cells. We further studied the dynamics in physiological conditions and analyzed the influence of the cell cycle on the circadian oscillator in tissues _in vivo_.

## Simplified model

Very briefly, the model for the phases of the two oscillators can be reduced as the following set of equations:

$$
\left\{
\begin{array}{l}
\textrm{d}\theta_t = \frac{2\pi} {T_\theta} \textrm{d}t + F(\theta_t,\phi_t) \textrm{d}t + \sigma_\theta\textrm{d} W_t \\
\textrm{d}\phi_t = \frac{2\pi} {T_\phi^i} \textrm{d}t 
\end{array}
\right.
$$

That is, we assume a linear, noiseless phase progression for $\phi$, which represents the cell-cycle, while the circadian clock phase can be impacted through a coupling function $F$, and is subject to noise through a Wiener process $W$.

From this phase model, one can generate a signal for the clock, using a new set of equation:

$$
\left\{
\begin{array}{l}
\textrm{d}A_t = -\gamma_{A} (A_t-\mu_{A}) \textrm{d}t + \sigma_{A}\textrm{d}W_t \\
\textrm{d}B_t = -\gamma_{B} (B_t-\mu_{B}) \textrm{d}t + \sigma_{B}\textrm{d}W_t \\
s_t = \exp(A_t) w(\theta_t) + B_t +\xi
\end{array}
\right.
$$

In practice, this means that we represent oscillations through a periodic function $w(\theta_t)$ (whose parameters must be optimized), while we correct for variations in amplitude and background using two Ornstein-Uhlenbeck (O-U) processes $A$ and $B$.

To provide a better ideau of what all these equations represent, we can simulate the uncoupled version (i.e. $F(\theta,\phi):=0$) of this basic model relatively easily, with the Euler method:

In [7]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from IPython.display import HTML
from matplotlib import animation

sn.set_style("white")
current_palette = sn.color_palette()

# Chose a sinusoidal function as the waveform
def waveform(theta, a, b, sigma=0.1):
    return ((np.cos(theta) + 1) / 2) ** 1.6 * a + b + np.random.normal(0, sigma)


# Define model parameters
TF = 48
N = TF * 2
tspan = np.concatenate(
    (np.linspace(0, TF, N), np.linspace(0, TF, N), np.linspace(0, TF, N), np.linspace(0, TF, N))
)
dt = TF / N
theta = 0
l_theta = [theta]
l_s = [waveform(theta, 1, 0)]
T_theta = 24.0
w = 2 * np.pi / T_theta
sigma = 0.05
a = 1.5
b = +0.5

# Simulate the system with a basic Euler method
for t in tspan[1:]:
    theta += w * dt + np.random.normal(0, dt) * sigma
    a += -1 / 35 * (a - 1) * dt + np.random.normal(0, dt) * sigma
    b += -1 / 35 * (b - 0.1) * dt + np.random.normal(0, dt) * sigma
    sig = 0.1
    l_theta.append(theta % (2 * np.pi))
    l_s.append(waveform(theta, a, b, sig))

# Mask the phase domain, to prevent discontinuites in the representation of the signal
abs_d_data_x = np.abs(np.diff(tspan))
mask_x = np.hstack([abs_d_data_x > abs_d_data_x.mean() + 3 * abs_d_data_x.std(), [False]])
masked_data_x = np.array([x if not m else np.nan for x, m in zip(tspan, mask_x)])
tspan = masked_data_x

# Build the figure
fig, ax = plt.subplots(figsize=(15, 5))
(line,) = ax.plot(tspan, l_s, color=current_palette[1])
(pts,) = ax.plot(tspan, l_s, "o", color=current_palette[1])
sn.despine(offset=5, trim=True)

# Function to update the data
def update(num, tspan, l_s, line, pts):
    line.set_data(tspan[max(0, num - 50) : num], l_s[max(0, num - 50) : num])
    pts.set_data(tspan[num - 1], l_s[num - 1])
    line.axes.axis([0, 48, -0.5, 2.5])
    return line, pts


# Animate
anim = animation.FuncAnimation(
    fig, update, frames=len(tspan), fargs=[tspan, l_s, line, pts], interval=30, blit=True
)
HTML(anim.to_html5_video())


The difficulty is then to optimize the parameters of this model on experimental traces. Explaining this in details would take pages and pages (see Supplementary Material of [the paper](https://www.nature.com/articles/s41567-019-0598-1)). But, very basically, we convert the continous variables into discrete ones, essentially turning our set of equations into a Hidden Markov Model (HMM), in which the phase of the circadian clock is a _latent variable_. We then optimize the parameters of this HMM through a probabilistic framework, using an Expectation-Maximization (EM) algorithm, whose update equations must be derived analytically, from the derivative of what is called the $Q$ function:

$$
Q(\mathbf{\Lambda},\mathbf{\Lambda}') = \sum_{\textbf{X} \in \boldsymbol{\mathcal{X}}} \text{log}(p(\textbf{O},\boldsymbol{X}|\mathbf{\Lambda}))p(\boldsymbol{X}|\textbf{O},\mathbf{\Lambda}')
$$

Where $\boldsymbol{\mathcal{X}}$ is the set of all possible states of the HMM (including phase and O-U processes states), and $\textbf{O}$ is the set of observations (the data).

The overall idea is that we can use the $Q$ function to optimize the parameters of the HMM, which themselves explain the true dynamics of the oscillators (in the low-dimensional phase-space). Don't hesitate to check the paper if you're interested!

## Artwork

Nature Physics offered us the possibility to submit an artwork as a potential cover for the magazine, along with the final paper. With the help of numerical simulations, Photoshop and Illustrator, I did my best to produce something remaining relatively sound scientifically, while being appealing to the audience. Unfortunately, it did not make it! But I still like this work very much. You can check the high-resolution version [here](https://colasdroin.github.io/images/artwork_coupling.png).

![image artwork](https://colasdroin.github.io/images/artwork_coupling_small.png)